In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import math
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib import cm, colors
import copy
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial import distance_matrix
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.progress import *
%matplotlib inline
palette = sns.color_palette("bright", 10)

In [2]:
#n=1000
#sampled_df1=pd.read_csv("dataset/sub_data_500_20.csv")
sampled_df1=pd.read_csv("C:/Users/Zhifeng Guo/python_code/paper3_code/Lagrange_Relaxation/LR_equality/multiple_randomized_datasets_2000points/dataset/sub_data_2000_1.csv")
n=sampled_df1.shape[0]
df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]
#df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]

In [3]:
df_X.shape

(2000, 150)

In [288]:
#k0=30
#3 to 17
#k=9
#kmeans = KMeans(n_clusters=k0, random_state=0).fit(df_X)
#cluster_centers=pd.DataFrame(kmeans.cluster_centers_,columns=df_X.columns, index=["center"+str(i) for i in range(k0)])).to_csv('sub_data_500_3_cluster_centers')

In [4]:
dfij=pd.DataFrame(distance_matrix(df_X.values, df_X.values), index=df_X.index, columns=df_X.index)
dij=dfij.to_numpy()
dij = np.round(dij, decimals=2)

In [5]:
dij.shape

(2000, 2000)

In [6]:
def label_transform(x):
    if x in (1, 2, 3):
        return 'H'
    if x in (52, 53, 54):
        return 'L'
    return 'O'

def label_to_color(x):
    if x in (1,2,3):
        return c4
    if x in (52,53,54):
        return c1
    return c5

def label_to_alphas(x):
    if x in (1,2,3):
        return 1
    if x in (52,53,54):
        return 1
    return 0.4

In [7]:
sampled_df1['plot'] = sampled_df1['ACORN_Type'].apply(lambda x:label_transform(x))
H=sampled_df1[sampled_df1['plot']=='H'].index
L=sampled_df1[sampled_df1['plot']=='L'].index
O=sampled_df1[sampled_df1['plot']=='O'].index

In [8]:
N=range(sampled_df1.shape[0])

In [9]:
N

range(0, 2000)

In [10]:
def CPLEX_Model(name,N,H,L,k,dij):
    mdl=Model(name=name)
        #set variables
    x=mdl.binary_var_dict([(i,j) for i in N for j in N],lb=0.0, name="x")
    y=mdl.binary_var_dict([j for j in N],name="y")
    mdl.add_constraints(x[i, j] -y[j]<= 0 for i in N for j in N)
    mdl.add_constraint(mdl.sum(y[j] for j in N) ==k)
    mdl.add_constraints(mdl.sum(x[i, j] for j in N) == 1 for i in N)
    mdl.add_constraints(x[i1, j]+x[i2, j] -1<= 0 for i1 in H for i2 in L for j in N)
    mdl.minimize(mdl.sum(dij[i, j]*x[i,j] for i in N for j in N))
    sol_model = mdl.solve(clean_before_solve=True)
    print("solved!")
    return x, y, sol_model

In [11]:
def get_y_values(y):
    Yj=[]
    for i in N:
        if y[i].solution_value==1:
            Yj.append(i)
    return Yj

In [12]:
k_list=[]
times=[]
objective_value=[]

In [13]:
for k in range(3,18):
    start2=datetime.now()
    x, y, sol_model=CPLEX_Model(name="CPLEX_Model",N=N,H=H,L=L,k=k,dij=dij)
    running_time=datetime.now()-start2
    cluster_points=get_y_values(y=y)
    pd.DataFrame(cluster_points, columns=['cluster_points']).to_csv('results/without_2000_5_cluster_points_'+str(k)+'_.csv')
    obj=sol_model.get_objective_value()
    objective_value.append(obj)
    times.append(running_time)
    k_list.append(k)
    print(running_time)

solved!


DOcplexException: Model<CPLEX_Model> did not solve successfully

In [299]:
pd.DataFrame(list(zip(k_list, times, objective_value)),
              columns=['k','Times','Objective_value']).to_csv('results/times_objective_value_without_variable_reduction_2000_5.csv')